In [13]:
import pandas as pd
import os

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [12]:
base_dir = "data/scraped/parsed/"
race_dir = f'{base_dir}round/'
split_dir = f'{base_dir}split/'

In [ ]:
for round_file in os.listdir(race_dir):
    # load in base round data
    round_df = pd.read_csv(os.path.join(race_dir, round_file))
    
    # enhance round data with information in filename
    round_metadata = round_file.split('-')
    season = f'{round_metadata[0]}-{round_metadata[1]}'
    competition = round_metadata[2]
    
    
    # load in relevant split data files
    
    # enhance split files with information in filename

In [7]:
race = pd.read_csv("data/scraped/parsed/round/2019-2020-ISUWorldJuniorShortTrackChampionships2020-Bormio(ITA)-500m-m-Heats.csv")
splits = pd.read_csv("data/scraped/parsed/split/2019-2020-ISUWorldJuniorShortTrackChampionships2020-Bormio(ITA)-500m-m-Heats-race_1-splits.csv")

In [8]:
race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   race        34 non-null     int64  
 1   Place       34 non-null     object 
 2   Start Pos.  34 non-null     int64  
 3   Warn.       0 non-null      float64
 4   #           34 non-null     int64  
 5   Name        34 non-null     object 
 6   ISU ID      34 non-null     int64  
 7   ISU Member  34 non-null     object 
 8   Results     34 non-null     object 
 9   Qual.       20 non-null     object 
dtypes: float64(1), int64(4), object(5)
memory usage: 2.8+ KB


In [11]:
splits.head(100)

,ZHANG Tianyi POSITION,ZHANG Tianyi LAP TIME,ZHANG Tianyi ELAPSED TIME,IVLIEV Konstantin POSITION,IVLIEV Konstantin LAP TIME,IVLIEV Konstantin ELAPSED TIME,Jens van 'T WOUT POSITION,Jens van 'T WOUT LAP TIME,Jens van 'T WOUT ELAPSED TIME,MIYATA Shogo POSITION,MIYATA Shogo LAP TIME,MIYATA Shogo ELAPSED TIME,Luca SPECHENHAUSER POSITION,Luca SPECHENHAUSER LAP TIME,Luca SPECHENHAUSER ELAPSED TIME
0,2,6.92,6.92,1,6.68,NaN,3,7.02,7.02,4,7.16,7.16,5,7.34,7.34
1,2,8.69,15.61,1,8.74,NaN,3,8.67,15.69,4,8.86,16.02,5,8.80,16.14
2,2,8.40,24.01,1,8.44,NaN,3,8.37,24.06,5,8.54,24.56,4,8.32,24.46
3,1,8.45,32.46,3,8.70,NaN,2,8.48,32.54,5,8.68,33.24,4,8.43,32.89
4,1,8.84,41.30,2,8.97,NaN,5,24.97,57.51,4,8.79,42.03,3,8.69,41.58


In [9]:
race.head(100)

,race,Place,Start Pos.,Warn.,#,Name,ISU ID,ISU Member,Results,Qual.
0,0,1,1,NaN,105,ZHANGTianyi,9713,CHN,41.303,Q
1,0,2,5,NaN,135,LucaSPECHENHAUSER,9398,ITA,41.585,Q
2,0,3,4,NaN,140,MIYATAShogo,9460,JPN,42.034,q
3,0,4,3,NaN,161,Jensvan'TWOUT,9551,NED,57.510,ADV
4,0,YC,2,NaN,169,IVLIEVKonstantin,9417,RUS,YC,NaN
5,1,1,2,NaN,168,BALBEKOVVladimir,9738,RUS,40.938,Q
6,1,2,1,NaN,104,SUNLong,9454,CHN,40.966,Q
7,1,3,3,NaN,160,NielsKINGMA,9552,NED,41.103,q
8,1,4,4,NaN,118,JUNGYanghunBen,9602,GER,42.315,NaN
9,1,5,5,NaN,175,DJORDJEVICJovan,9295,SRB,43.684,NaN
